```
library(keras)

# load the parameters
source("parameters.R")

# load the neural network model
model <- load_model_hdf5("model.h5", compile=FALSE)


# a function that generates a single pet name from a model
generate_name <- function(model, character_lookup, max_length, temperature=1){
  # model - the trained neural network
  # character_lookup - the table for how characters convert to numbers
  # max_length - the expected length of the training data in characters
  # temperature - how weird to make the names, higher is weirder
  
  # given the probabilities returned from the model, this code
  choose_next_char <- function(preds, character_lookup,temperature = 1){
    preds <- log(preds)/temperature
    exp_preds <- exp(preds)
    preds <- exp_preds/sum(exp(preds))
    
    next_index <- which.max(as.integer(rmultinom(1, 1, preds)))
    character_lookup$character[next_index-1]
  }
  
  in_progress_name <- character(0)
  next_letter <- ""
  
  # while we haven't hit a stop character and the name isn't too long
  while(next_letter != "+" && length(in_progress_name) < 30){
    # prep the data to run in the model again
    previous_letters_data <- 
      lapply(list(in_progress_name), function(.x){
        character_lookup$character_id[match(.x,character_lookup$character)]
      })
    previous_letters_data <- pad_sequences(previous_letters_data, maxlen = max_length)
    previous_letters_data <- to_categorical(previous_letters_data, num_classes = num_characters)
    
    # get the probabilities of each possible next character by running the model
    next_letter_probabilities <- 
      predict(model,previous_letters_data)
    
    # determine what the actual letter is
    next_letter <- choose_next_char(next_letter_probabilities,character_lookup,temperature)
    
    if(next_letter != "+")
      # if the next character isn't stop add the latest generated character to the name and continue
      in_progress_name <- c(in_progress_name,next_letter)
  }
  
  # turn the list of characters into a single string
  raw_name <- paste0(in_progress_name, collapse="")
  
  # capitalize the first letter of each word
  capitalized_name <-gsub("\\b(\\w)","\\U\\1",raw_name,perl=TRUE)
  
  capitalized_name
}

# a function to generate many names
generate_many_names <- function(n=10, model, character_lookup, max_length, temperature=1){
  # n - the number of names to generate
  # (then everything else you'd pass to generate_name)
  unlist(lapply(1:n,function(x) generate_name(model, character_lookup, max_length, temperature)))
}
```

In [3]:
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import itertools
import random
import string
import io

In [4]:
character_list = list(string.ascii_lowercase) + [".","-"," ","+"]
character_lookup = dict(zip(character_list, range(len(character_list))))
max_length = 10
num_characters = len(character_lookup)

In [5]:
model = keras.models.load_model("model.h5")

In [54]:
def generate_name(model, character_lookup, max_length, temperature = 1):
    def choose_next_char(preds, character_lookup, temperature = 1):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="divide by zero encountered in divide")
            preds = np.log(preds)/temperature
        exp_preds = np.exp(preds)
        preds = exp_preds/sum(np.exp(preds))

        next_index = np.argmax(np.random.multinomial(1,preds))
        next_character = list(character_lookup.keys())[next_index]
        return next_character
    def characters_to_matrix(character_data):
        character_data = [[character_lookup[chr] for chr in c] for c in character_data]
        padded_character_data = keras.utils.pad_sequences(character_data, maxlen = max_length)
        text_matrix = keras.utils.to_categorical(padded_character_data, num_classes = num_characters)
        return text_matrix

    in_progress_name = []
    next_letter = ""
  
    while next_letter != "+" and len(in_progress_name) < 30:
        previous_letters_data = characters_to_matrix([in_progress_name])
        next_letter_probabilities = model.predict(previous_letters_data, verbose = 0)[0]
        next_letter = choose_next_char(next_letter_probabilities, character_lookup, temperature)
        if next_letter != "+":
            in_progress_name = in_progress_name + [next_letter]
    
    raw_name = ''.join(in_progress_name)
    capitalized_name = string.capwords(raw_name) 
  
    return capitalized_name

def generate_many_names(model, character_lookup, max_length, temperature = 1, n = 10):
    names = [generate_name(model, character_lookup, max_length, temperature) for i in range(n)]
    return names

In [55]:
generate_many_names(model, character_lookup, max_length, n = 20)

C:\Users\jacqu\AppData\Local\Temp\ipykernel_9640\792214307.py:3: RuntimeWarning: divide by zero encountered in log
  preds = np.log(preds)/temperature


['Amiji',
 'Baiy',
 'Laxie',
 'Jack',
 'Coaky',
 'Tilly',
 'Pepper',
 'Hubby',
 'Isis',
 'Milo',
 'Millis',
 'Yczie',
 'Della',
 'Katie',
 'Violet',
 'Mixie',
 'Hoover',
 'Misty Mil',
 'Maggie',
 'Finn']